In [1]:
import torch
import torch.nn as nn
import os

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
import numpy as np

from scipy.stats import spearmanr
import csv

In [2]:
def read_nodes(file_path):
    int_list = []
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            for item in row:
                try:
                    int_list.append(int(item))
                except ValueError:
                    print(f"Warning: '{item}' could not be converted to an integer and was skipped.")
    return int_list

def calculate_one(path):
    # score = torch.load(path, map_location=torch.device('cpu'))  # _test_0225_regroup
    score = torch.load(path, map_location=torch.device('cpu'))
    # score = torch.rand(5000, 500)
    print("score shape:", score.shape)

    nodes_str = []
    for i in range(50):
        nodes_str.append(f"./checkpoints/{i}/train_index.csv")

    full_nodes = [i for i in range(4656)]

    node_list = []
    for node_str in nodes_str:
        numbers = read_nodes(node_str)
        index = []
        for number in numbers:
            index.append(full_nodes.index(number))
        node_list.append(index)

    loss_list = torch.load("./result/gt.pt", map_location=torch.device('cpu')).detach()

    approx_output = []
    for i in range(len(nodes_str)):
        score_approx_0 = score[node_list[i], :]
        sum_0 = torch.sum(score_approx_0, axis=0)
        approx_output.append(sum_0)

    print(len(loss_list), loss_list[0].shape)
    print(len(approx_output), approx_output[0].shape)

    res = 0
    counter = 0
    for i in range(score.shape[1]):
        tmp = spearmanr(np.array([approx_output[k][i] for k in range(len(approx_output))]),
                        np.array([loss_list[k][i].numpy() for k in range(len(loss_list))])).statistic
        if np.isnan(tmp):
            print("Numerical issue")
            continue
        res += tmp
        counter += 1

    print(counter)

    return res/counter, loss_list, approx_output


In [3]:
for dim in [4096, 2048, 1024, 512]:
    path = f"./result/score_GIP-default_SJLT-{dim}.pt"
    print(dim, calculate_one(path)[0])

score shape: torch.Size([4656, 481])
50 torch.Size([481])
50 torch.Size([481])
481
4096 -0.014090898229616983
score shape: torch.Size([4656, 481])
50 torch.Size([481])
50 torch.Size([481])
481
2048 -0.022028237305852304
score shape: torch.Size([4656, 481])
50 torch.Size([481])
50 torch.Size([481])
481
1024 -0.019091281926670076
score shape: torch.Size([4656, 481])
50 torch.Size([481])
50 torch.Size([481])
481
512 -0.0035301664831030406


In [5]:
path = f"./result/score_GD-default_SJLT-2048.pt"
print(dim, calculate_one(path)[0])

score shape: torch.Size([4656, 481])
50 torch.Size([481])
50 torch.Size([481])
481
512 -0.037580277596140625
